In [1]:

import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import os
import re
import string
import json
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer

import nltk
os.makedirs(r'C:\\nltk_data', exist_ok=True)
nltk.data.path.append(r'C:\\nltk_data')  
nltk.download('punkt_tab', download_dir=r'C:\\nltk_data')
nltk.download('stopwords', download_dir=r'C:\\nltk_data')
nltk.download('wordnet', download_dir=r'C:\\nltk_data')

from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import SVC
from sklearn.metrics import classification_report, accuracy_score
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import make_pipeline

from transformers import AutoTokenizer, AutoModelForSequenceClassification
from torch.utils.data import Dataset, DataLoader
import torch
import torch.nn.functional as F
from tqdm import tqdm

[nltk_data] Downloading package punkt_tab to C:\\nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package stopwords to C:\\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to C:\\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
c:\Users\Lenovo\AppData\Local\Programs\Python\Python312\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
import pandas as pd

def read_database_result_csv(file_path):
    try:
        # Mencoba membaca dengan encoding default (utf-8) dan delimiter koma
        df = pd.read_csv(file_path, sep=';', encoding='utf-8')
    except UnicodeDecodeError as e1:
        try:
            # Jika gagal, mencoba dengan encoding latin1
            df = pd.read_csv(file_path, sep=';', encoding='latin1')
        except UnicodeDecodeError as e2:
            try:
                # Jika gagal, mencoba dengan encoding cp1252
                df = pd.read_csv(file_path, sep=';', encoding='cp1252')
            except Exception as e3:
                print(f"Gagal membaca file {file_path}: {e1} | {e2} | {e3}")
                return pd.DataFrame()  # Mengembalikan DataFrame kosong jika gagal
    return df

# Path ke file DATABASE_RESULT_OJK.csv
file_path = 'data/main_data_17.csv'
database_result_df = read_database_result_csv(file_path)

print(database_result_df)

# import pandas as pd

# def read_database_result_excel(file_path):
#     try:
#         # Membaca file Excel
#         df = pd.read_excel(file_path, engine='openpyxl')
#     except FileNotFoundError:
#         print(f"File tidak ditemukan: {file_path}")
#         return pd.DataFrame()  # Mengembalikan DataFrame kosong jika file tidak ditemukan
#     except Exception as e:
#         print(f"Gagal membaca file Excel {file_path}: {e}")
#         return pd.DataFrame()  # Mengembalikan DataFrame kosong jika gagal membaca
#     return df

# # Path ke file DATABASE_RESULT_OJK.xlsx
# file_path = 'DATABASE BESAR OJK X FINTECH UNS.xlsx'
# database_result_df = read_database_result_excel(file_path)

# print(database_result_df)



                      Dataset                      ID BIDANG SATKER (AKRONIM)  \
0           Database Internal    DATABASE INTERNAL_23     MS             DPSI   
1           Database Internal    DATABASE INTERNAL_24     MS             DPSI   
2           Database Internal    DATABASE INTERNAL_26     MS             DPSI   
3           Database Internal    DATABASE INTERNAL_28     MS             DPSI   
4           Database Internal    DATABASE INTERNAL_30     MS             DPSI   
...                       ...                     ...    ...              ...   
2916  DATABASE LOGIC INTERNAL  DATABASE INTERNAL_9528     MS             DPSU   
2917  DATABASE LOGIC INTERNAL  DATABASE INTERNAL_9537     MS             DPSU   
2918  DATABASE LOGIC INTERNAL  DATABASE INTERNAL_9540     MS             DPSU   
2919  DATABASE LOGIC INTERNAL  DATABASE INTERNAL_9637    ARK             DPAI   
2920  DATABASE LOGIC INTERNAL  DATABASE INTERNAL_9647    ARK             DPAI   

     JENIS SURVEI  TIPE QUE

In [ ]:
# #FILTERING SATKER

# # Menentukan filter SATKER yang diinginkan
# filter_option = input("Apakah Anda ingin memilih SATKER berdasarkan pilihan tertentu (y/n)? ")

# if filter_option.lower() == 'y':
#     # Jika pengguna memilih untuk memasukkan SATKER
#     selected_satker = input("Masukkan SATKER yang ingin difilter (misal: 'DPEA'): ")
    
#     # Memfilter data berdasarkan SATKER yang dipilih
#     filtered_df = database_result_df[database_result_df['SATKER (AKRONIM)'] == selected_satker]
#     print(f"Data difilter berdasarkan SATKER: {selected_satker}")
    
# elif filter_option.lower() == 'n':
#     # Jika pengguna memilih untuk memfilter berdasarkan beberapa SATKER tertentu
#     selected_satker_list = ['DPEA', 'DOSB', 'DPSI']  # Contoh daftar SATKER yang bisa dipilih
#     filtered_df = database_result_df[database_result_df['SATKER (AKRONIM)'].isin(selected_satker_list)]
#     print(f"Data difilter berdasarkan beberapa SATKER: {', '.join(selected_satker_list)}")
    
# else:
#     # Jika input selain 'y' atau 'n', tampilkan pesan
#     print("Pilihan tidak valid. Menggunakan data tanpa filter.")
#     filtered_df = database_result_df

# # Menampilkan data yang sudah difilter
# print(filtered_df)

In [ ]:
# print(database_result_df.isnull().sum())

In [ ]:
# drop = database_result_df.drop(index=0)

In [ ]:
# print(drop)

In [3]:
columns_order = [
    'ID','BIDANG','SATKER (AKRONIM)','JENIS SURVEI','TIPE QUESTION','INSTITUSI / PERSEORANGAN/ASAL SATKER','RESPOND','LINK SURVEYMONKEY','TOKEN','NAMA PIC/RESPONDEN','JABATAN/PROFESI/LVEL DI OJK','KONTAK','FUNGSI YANG DINILAI','DIRECT / INDIRECT','JENIS STAKEHOLDERS','RELASI RESPONDEN DENGAN SATKER','POWER','INTEREST','KATEGORI','Dataset','RESOURCE PERCEPTION','PERFORMANCE DELIVERY','OPEN QUESTION 1','OPEN QUESTION 2'
]

In [4]:
all_data_idi_df = database_result_df[columns_order]

In [5]:
print(all_data_idi_df)

                          ID BIDANG SATKER (AKRONIM) JENIS SURVEI  \
0       DATABASE INTERNAL_23     MS             DPSI     INTERNAL   
1       DATABASE INTERNAL_24     MS             DPSI     INTERNAL   
2       DATABASE INTERNAL_26     MS             DPSI     INTERNAL   
3       DATABASE INTERNAL_28     MS             DPSI     INTERNAL   
4       DATABASE INTERNAL_30     MS             DPSI     INTERNAL   
...                      ...    ...              ...          ...   
2916  DATABASE INTERNAL_9528     MS             DPSU     INTERNAL   
2917  DATABASE INTERNAL_9537     MS             DPSU     INTERNAL   
2918  DATABASE INTERNAL_9540     MS             DPSU     INTERNAL   
2919  DATABASE INTERNAL_9637    ARK             DPAI     INTERNAL   
2920  DATABASE INTERNAL_9647    ARK             DPAI     INTERNAL   

      TIPE QUESTION INSTITUSI / PERSEORANGAN/ASAL SATKER       RESPOND  \
0       DIRECT DPSI                                 DPAI  SUDAH DI ISI   
1       DIRECT DPSI    

In [6]:
print(all_data_idi_df.isnull().sum())

ID                                       0
BIDANG                                   0
SATKER (AKRONIM)                         0
JENIS SURVEI                             0
TIPE QUESTION                            0
INSTITUSI / PERSEORANGAN/ASAL SATKER     0
RESPOND                                  0
LINK SURVEYMONKEY                        0
TOKEN                                    0
NAMA PIC/RESPONDEN                       0
JABATAN/PROFESI/LVEL DI OJK              0
KONTAK                                   0
FUNGSI YANG DINILAI                      0
DIRECT / INDIRECT                        0
JENIS STAKEHOLDERS                       0
RELASI RESPONDEN DENGAN SATKER           0
POWER                                    0
INTEREST                                 0
KATEGORI                                 0
Dataset                                  0
RESOURCE PERCEPTION                     60
PERFORMANCE DELIVERY                    60
OPEN QUESTION 1                          6
OPEN QUESTI

In [7]:
all_data_idi_df.fillna("-", inplace=True)

print(all_data_idi_df.isnull().sum())

ID                                      0
BIDANG                                  0
SATKER (AKRONIM)                        0
JENIS SURVEI                            0
TIPE QUESTION                           0
INSTITUSI / PERSEORANGAN/ASAL SATKER    0
RESPOND                                 0
LINK SURVEYMONKEY                       0
TOKEN                                   0
NAMA PIC/RESPONDEN                      0
JABATAN/PROFESI/LVEL DI OJK             0
KONTAK                                  0
FUNGSI YANG DINILAI                     0
DIRECT / INDIRECT                       0
JENIS STAKEHOLDERS                      0
RELASI RESPONDEN DENGAN SATKER          0
POWER                                   0
INTEREST                                0
KATEGORI                                0
Dataset                                 0
RESOURCE PERCEPTION                     0
PERFORMANCE DELIVERY                    0
OPEN QUESTION 1                         0
OPEN QUESTION 2                   

C:\Users\Lenovo\AppData\Local\Temp\ipykernel_38508\4226600997.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  all_data_idi_df.fillna("-", inplace=True)


In [ ]:
all_data_idi_df.describe()

In [8]:
def preprocess_text(text):
    text = text.lower()
    text = re.sub(r'http\S+|www\S+|https\S+|\@\w+|\#|\d+|[^\w\s]', '', text)
    text = re.sub(r'\s+', ' ', text).strip()
    text = text.translate(str.maketrans('', '', string.punctuation))
    words = word_tokenize(text)
    stop_words = set(stopwords.words('indonesian'))
    words = [word for word in words if word not in stop_words]
    lemmatizer = WordNetLemmatizer()
    words = [lemmatizer.lemmatize(word) for word in words]
    cleaned_text = ' '.join(words)
    
    return cleaned_text

In [ ]:
# import json
# from textblob import  TextBlob
# #VALIDASI

# file_path = 'data/hasil/main_data_idi.csv'

# # Load dataset
# data = pd.read_csv(file_path, delimiter=';')

# data_copy = data.copy()

# # Fungsi untuk melakukan labeling berdasarkan polaritas
# def assign_label(text):
#     # Membuat objek TextBlob dari teks
#     blob = TextBlob(text)
    
#     # Menghitung polaritas
#     polarity = blob.sentiment.polarity
    
#     # Menentukan label berdasarkan polaritas
#     if polarity > 0.5:
#         return "Sangat Setuju"
#     elif polarity > 0:
#         return "Setuju"
#     elif polarity == 0:
#         return "Cukup Setuju"
#     elif polarity < -0.5:
#         return "Sangat Tidak Setuju"
#     else:
#         return "Kurang Setuju"


# # Membaca file JSON yang berisi mapping
# with open('mapping.json', 'r') as file:
#     text_mapping = json.load(file)


# # # Fungsi untuk menentukan label berdasarkan kata-kata dalam 'Combined_Text'
# def map_combined_text_to_label(text):
#     # Iterasi untuk mencari kata kunci dalam text
#     for keyword, label in text_mapping.items():
#         if keyword in text.lower():  # Mengabaikan kapitalisasi
#             return label
#     return 'setuju'  # Jika tidak ada kata kunci yang ditemukan

# # Terapkan fungsi ke kolom 'Combined_Text' untuk membuat kolom 'Label'
# data_copy['New_Label'] = data_copy['Combined_Text'].apply(assign_label)

# # Tampilkan dataset yang sudah dimodifikasi
# data_copy['Label'] = data_copy['New_Label'].combine_first(data_copy['Label'])

# # Simpan dataset yang sudah dimodifikasi
# # data_copy.to_csv('data/hasil/validated_dataset_new.csv', index=False, sep=';')

# Menampilkan statistik label setelah perubahan
# label_counts = data_copy['Label'].value_counts()
# label_summary = pd.DataFrame(label_counts).reset_index()
# label_summary.columns = ['Label', 'Count']

# label_map = {
#     1: "sangat tidak setuju",
#     2: "tidak setuju",
#     3: "kurang setuju",
#     4: "cukup setuju",
#     5: "setuju",
#     6: "sangat setuju"
# }

# reverse_label_map = {v: k for k, v in label_map.items()}  

# data_copy['Label_Index'] = data_copy['Label'].map(reverse_label_map)

# def calculate_weight(index):
#     if index == 1:
#         return 1
#     elif 1.1 <= index <= 2.9:
#         return 2.95
#     elif 3 <= index <= 3.9:
#         return 5.9
#     elif 4 <= index <= 6:
#         return 6
#     else:
#         return None  

# data_copy['NILAI_SENTIMEN'] = data_copy['Label_Index'].apply(calculate_weight)

# # print(data_copy)


# print(data_copy[['Label','NILAI_SENTIMEN']])
# data_copy.to_csv('data/hasil/main_data_idi.csv', index=False, sep=';')

# label_counts = data_copy['Label'].value_counts()
# label_summary = pd.DataFrame(label_counts).reset_index()
# label_summary.columns = ['Label', 'Count']

In [9]:
# from transformers import AutoTokenizer, AutoModelForSequenceClassification
# import torch
# import torch.nn.functional as F
# import pandas as pd
# from tqdm import tqdm

# # Ganti model_name dengan model IndoBERT
# model_name = "indobenchmark/indobert-base-p1"
# tokenizer = AutoTokenizer.from_pretrained(model_name)
# model = AutoModelForSequenceClassification.from_pretrained(
#     model_name,
#     num_labels=6  # Sesuaikan jumlah label
# )

# label_map = {
#     0: "sangat tidak setuju",
#     1: "tidak setuju",
#     2: "kurang setuju",
#     3: "cukup setuju",
#     4: "setuju",
#     5: "sangat setuju"
# }

# # Fungsi untuk memproses teks
# def preprocess_text(text):
#     return text.strip().lower() if isinstance(text, str) else ""  # Periksa apakah teks valid

# # Fungsi untuk prediksi sentimen
# def predict_sentiment(texts):
#     model.eval()
#     results = []
    
#     with torch.no_grad():
#         for text in tqdm(texts):
#             text = preprocess_text(text)
            
#             if not text:  # Lewati teks kosong
#                 results.append({
#                     'text': text,
#                     'sentiment': "unknown",
#                     'confidence': 0.0
#                 })
#                 continue
            
#             # Tokenisasi teks
#             encoded = tokenizer(
#                 text,
#                 truncation=True,
#                 padding=True,
#                 max_length=512,
#                 return_tensors="pt"
#             )
            
#             # Model memprediksi
#             outputs = model(encoded["input_ids"], attention_mask=encoded["attention_mask"])
#             predictions = F.softmax(outputs.logits, dim=1)
#             predicted_label = torch.argmax(predictions, dim=1).item()
            
#             # Ambil confidence score
#             confidence = predictions[0][predicted_label].item()
            
#             # Mapping hasil prediksi ke label
#             sentiment = label_map.get(predicted_label, "unknown")
            
#             # Menambahkan hasil ke dalam list
#             results.append({
#                 'text': text,
#                 'sentiment': sentiment,
#                 'confidence': confidence
#             })
    
#     return pd.DataFrame(results)

# columns_to_process = ['OPEN QUESTION 1','OPEN QUESTION 2']

# all_data_idi_df['Text'] = all_data_idi_df[columns_to_process].fillna("").apply(lambda row: " ".join(row), axis=1)

# results = predict_sentiment(all_data_idi_df['Text'].tolist())

# all_data_idi_df.loc[:, 'Label'] = results['sentiment']
# all_data_idi_df.loc[:, 'Confidence'] = results['confidence']

# # Menampilkan hasil
# print("\nSample of labeled data:")
# print(all_data_idi_df[['OPEN QUESTION 1','OPEN QUESTION 2','Label', 'Confidence']].head())


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at indobenchmark/indobert-base-p1 and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
C:\Users\Lenovo\AppData\Local\Temp\ipykernel_30824\1506450411.py:76: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  all_data_idi_df['Text'] = all_data_idi_df[columns_to_process].fillna("").apply(lambda row: " ".join(row), axis=1)
100%|██████████| 2921/2921 [04:12<00:00, 11.58it/s]


Sample of labeled data:
                                     OPEN QUESTION 1  \
0  Sdh sangat bagus hanya banyak terkendala krn k...   
1                                        sangat baik   
2                            sudah baik, pertahankan   
3                     Sejauh ini baik dan responsif.   
4  Pengelolaan sistem informasi oleh DPSI OJK tel...   

                                     OPEN QUESTION 2         Label  Confidence  
0  fungsi pengelolaan informasi harus imbang anta...  tidak setuju    0.280958  
1                                                  -  tidak setuju    0.194074  
2  sudah cukup baik pertahankan kinerja yang suda...  tidak setuju    0.254202  
3  Agar terdapat no wa helpdesk per masing-masing...  tidak setuju    0.283238  
4  agar dapat setiap pegawai di fasilitas dengan ...  tidak setuju    0.235132  



C:\Users\Lenovo\AppData\Local\Temp\ipykernel_30824\1506450411.py:80: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  all_data_idi_df.loc[:, 'Label'] = results['sentiment']
C:\Users\Lenovo\AppData\Local\Temp\ipykernel_30824\1506450411.py:81: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  all_data_idi_df.loc[:, 'Confidence'] = results['confidence']


In [9]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch
import torch.nn.functional as F
import pandas as pd
from tqdm import tqdm

# Ganti model_name dengan model IndoBERT
model_name = "indobenchmark/indobert-base-p1"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(
    model_name,
    num_labels=6  # Sesuaikan jumlah label
)

label_map = {
    0: "sangat tidak setuju",
    1: "tidak setuju",
    2: "kurang setuju",
    3: "cukup setuju",
    4: "setuju",
    5: "sangat setuju"
}

# Fungsi untuk memproses teks
def preprocess_text(text):
    return text.strip().lower() if isinstance(text, str) else ""  # Periksa apakah teks valid

# Fungsi untuk prediksi sentimen
def predict_sentiment(texts):
    model.eval()
    results = []
    
    with torch.no_grad():
        for text in tqdm(texts):
            text = preprocess_text(text)
            
            if not text:  # Lewati teks kosong
                results.append({
                    'text': text,
                    'sentiment': "unknown",
                    'confidence': 0.0
                })
                continue
            
            # Tokenisasi teks
            encoded = tokenizer(
                text,
                truncation=True,
                padding=True,
                max_length=512,
                return_tensors="pt"
            )
            
            # Model memprediksi
            outputs = model(encoded["input_ids"], attention_mask=encoded["attention_mask"])
            predictions = F.softmax(outputs.logits, dim=1)
            predicted_label = torch.argmax(predictions, dim=1).item()
            
            # Ambil confidence score
            confidence = predictions[0][predicted_label].item()
            
            # Mapping hasil prediksi ke label
            sentiment = label_map.get(predicted_label, "unknown")
            
            # Menambahkan hasil ke dalam list
            results.append({
                'text': text,
                'sentiment': sentiment,
                'confidence': confidence
            })
    
    return pd.DataFrame(results)

columns_to_process = ['OPEN QUESTION 1']

all_data_idi_df['Text'] = all_data_idi_df[columns_to_process].fillna("").apply(lambda row: " ".join(row), axis=1)

results = predict_sentiment(all_data_idi_df['Text'].tolist())

all_data_idi_df.loc[:, 'Label'] = results['sentiment']
all_data_idi_df.loc[:, 'Confidence'] = results['confidence']

# Menampilkan hasil
print("\nSample of labeled data:")
print(all_data_idi_df[['OPEN QUESTION 1','Label', 'Confidence']].head())


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at indobenchmark/indobert-base-p1 and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
C:\Users\Lenovo\AppData\Local\Temp\ipykernel_38508\3337752218.py:76: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  all_data_idi_df['Text'] = all_data_idi_df[columns_to_process].fillna("").apply(lambda row: " ".join(row), axis=1)
100%|██████████| 2921/2921 [03:49<00:00, 12.70it/s]


Sample of labeled data:
                                     OPEN QUESTION 1          Label  \
0  Sdh sangat bagus hanya banyak terkendala krn k...         setuju   
1                                        sangat baik         setuju   
2                            sudah baik, pertahankan   cukup setuju   
3                     Sejauh ini baik dan responsif.   cukup setuju   
4  Pengelolaan sistem informasi oleh DPSI OJK tel...  kurang setuju   

   Confidence  
0    0.222106  
1    0.199094  
2    0.208029  
3    0.213246  
4    0.229381  



C:\Users\Lenovo\AppData\Local\Temp\ipykernel_38508\3337752218.py:80: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  all_data_idi_df.loc[:, 'Label'] = results['sentiment']
C:\Users\Lenovo\AppData\Local\Temp\ipykernel_38508\3337752218.py:81: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  all_data_idi_df.loc[:, 'Confidence'] = results['confidence']


In [10]:
all_data_idi_df.to_csv('data/hasil/main_data_17_OQ1.csv', index=False, sep=';')

In [11]:
print(all_data_idi_df)

                          ID BIDANG SATKER (AKRONIM) JENIS SURVEI  \
0       DATABASE INTERNAL_23     MS             DPSI     INTERNAL   
1       DATABASE INTERNAL_24     MS             DPSI     INTERNAL   
2       DATABASE INTERNAL_26     MS             DPSI     INTERNAL   
3       DATABASE INTERNAL_28     MS             DPSI     INTERNAL   
4       DATABASE INTERNAL_30     MS             DPSI     INTERNAL   
...                      ...    ...              ...          ...   
2916  DATABASE INTERNAL_9528     MS             DPSU     INTERNAL   
2917  DATABASE INTERNAL_9537     MS             DPSU     INTERNAL   
2918  DATABASE INTERNAL_9540     MS             DPSU     INTERNAL   
2919  DATABASE INTERNAL_9637    ARK             DPAI     INTERNAL   
2920  DATABASE INTERNAL_9647    ARK             DPAI     INTERNAL   

      TIPE QUESTION INSTITUSI / PERSEORANGAN/ASAL SATKER       RESPOND  \
0       DIRECT DPSI                                 DPAI  SUDAH DI ISI   
1       DIRECT DPSI    

In [ ]:
# # Menyimpan data ke file CSV
# output_file = "./data/hasil/all_data_idi_ss.csv"
# all_data_idi_df.to_csv(output_file, index=False, sep=';', encoding='utf-8-sig')

# print(f"Data berhasil disimpan ke {output_file}")

In [ ]:
# #SAVE MODEL

# from transformers import AutoTokenizer, AutoModelForSequenceClassification

# # Ganti model_name dengan model IndoBERT
# model_name = "indobenchmark/indobert-base-p1"
# tokenizer = AutoTokenizer.from_pretrained(model_name)
# model = AutoModelForSequenceClassification.from_pretrained(
#     model_name,
#     num_labels=6  # Sesuaikan jumlah label
# )

# # Menyimpan model dan tokenizer ke direktori
# model_save_path = './saved_model'
# tokenizer.save_pretrained(model_save_path)
# model.save_pretrained(model_save_path)

# print(f"Model dan tokenizer berhasil disimpan di {model_save_path}")

In [ ]:
# #LOAD MODEL

# from transformers import AutoTokenizer, AutoModelForSequenceClassification

# # Path ke model yang disimpan
# model_save_path = './saved_model'

# # Memuat model dan tokenizer dari path yang telah disimpan
# tokenizer = AutoTokenizer.from_pretrained(model_save_path)
# model = AutoModelForSequenceClassification.from_pretrained(model_save_path)

# print(f"Model dan tokenizer berhasil dimuat dari {model_save_path}")

In [12]:
label_counts = all_data_idi_df['Label'].value_counts()

label_summary = pd.DataFrame(label_counts).reset_index()
label_summary.columns = ['Label', 'Count']
print(label_summary)

                 Label  Count
0               setuju   1760
1        kurang setuju    768
2         cukup setuju    275
3        sangat setuju    116
4  sangat tidak setuju      2


In [ ]:
# os.makedirs('data/hasil', exist_ok=True)
# all_data_idi_df.to_csv('data/hasil/labeled.csv', index=False, sep=";")

In [ ]:
# import json
# from textblob import TextBlob
# #VALIDASI

# file_path = 'data/hasil/main_data_REV.csv'

# # Load dataset
# data = pd.read_csv(file_path, delimiter=';')

# data_copy = data.copy()

# # Fungsi untuk melakukan labeling berdasarkan polaritas
# def assign_label(text):
#     # Membuat objek TextBlob dari teks
#     blob = TextBlob(text)
    
#     # Menghitung polaritas
#     polarity = blob.sentiment.polarity
    
#     # Menentukan label berdasarkan polaritas
#     if polarity > 0.5:
#         return "Sangat Setuju"
#     elif polarity > 0:
#         return "Setuju"
#     elif polarity == 0:
#         return "Cukup Setuju"
#     elif polarity < -0.5:
#         return "Sangat Tidak Setuju"
#     else:
#         return "Kurang Setuju"


# # # Membaca file JSON yang berisi mapping
# with open('mapping.json', 'r') as file:
#     text_mapping = json.load(file)


# # # Fungsi untuk menentukan label berdasarkan kata-kata dalam 'Combined_Text'
# def map_combined_text_to_label(text):
#     # Iterasi untuk mencari kata kunci dalam text
#     for keyword, label in text_mapping.items():
#         if keyword in text.lower():  # Mengabaikan kapitalisasi
#             return label
#     return 'setuju'  # Jika tidak ada kata kunci yang ditemukan

# # Terapkan fungsi ke kolom 'Combined_Text' untuk membuat kolom 'Label'
# data_copy['New_Label'] = data_copy['Text'].apply(assign_label)

# # Tampilkan dataset yang sudah dimodifikasi
# data_copy['Label'] = data_copy['New_Label'].combine_first(data_copy['Label'])

# # Simpan dataset yang sudah dimodifikasi
# data_copy.to_csv('data/hasil/validated_dataset_new.csv', index=False, sep=';')

# # Menampilkan statistik label setelah perubahan
# label_counts = data_copy['Label'].value_counts()
# label_summary = pd.DataFrame(label_counts).reset_index()
# label_summary.columns = ['Label', 'Count']

# label_map = {
#     1: "sangat tidak setuju",
#     2: "tidak setuju",
#     3: "kurang setuju",
#     4: "cukup setuju",
#     5: "setuju",
#     6: "sangat setuju"
# }

# reverse_label_map = {v: k for k, v in label_map.items()}  

# data_copy['Label_Index'] = data_copy['Label'].map(reverse_label_map)

# def calculate_weight(index):
#     if index == 1:
#         return 1
#     elif 1.1 <= index <= 2.9:
#         return 2.95
#     elif 3 <= index <= 3.9:
#         return 5.9
#     elif 4 <= index <= 6:
#         return 6
#     else:
#         return None  

# data_copy['NILAI_SENTIMEN'] = data_copy['Label_Index'].apply(calculate_weight)

# # print(data_copy)


# print(data_copy[['Label','NILAI_SENTIMEN']])
# data_copy.to_csv('data/hasil/main_data_REVREV.csv', index=False, sep=';')

# label_counts = data_copy['Label'].value_counts()
# label_summary = pd.DataFrame(label_counts).reset_index()
# label_summary.columns = ['Label', 'Count']

In [13]:
# TODO: ini yang bener

import json
#VALIDASI

file_path = 'data/hasil/main_data_17_OQ1.csv'

# Load dataset
data = pd.read_csv(file_path, delimiter=';')

data_copy = data.copy()

# Membaca file JSON yang berisi mapping
with open('mapping.json', 'r') as file:
    text_mapping = json.load(file)
    
# data_copy['Text'] = data_copy['Text'].str.lower()

# Fungsi untuk menentukan label berdasarkan kata-kata dalam 'Combined_Text'
def map_combined_text_to_label(text):
    # Iterasi untuk mencari kata kunci dalam text
    for keyword, label in text_mapping.items():
        if keyword in text.lower():  # Mengabaikan kapitalisasi
            return label
    return 'setuju'  # Jika tidak ada kata kunci yang ditemukan

# Terapkan fungsi ke kolom 'Combined_Text' untuk membuat kolom 'Label'
data_copy['New_Label'] = data_copy['Text'].apply(map_combined_text_to_label)

# Tampilkan dataset yang sudah dimodifikasi
data_copy['Label'] = data_copy['New_Label'].combine_first(data_copy['Label'])

# Simpan dataset yang sudah dimodifikasi
data_copy.to_csv('data/hasil/validated_dataset_new.csv', index=False, sep=';')

# Menampilkan statistik label setelah perubahan
label_counts = data_copy['Label'].value_counts()
label_summary = pd.DataFrame(label_counts).reset_index()
label_summary.columns = ['Label', 'Count']

label_map = {
    1: "sangat tidak setuju",
    2: "tidak setuju",
    3: "kurang setuju",
    4: "cukup setuju",
    5: "setuju",
    6: "sangat setuju"
}

reverse_label_map = {v: k for k, v in label_map.items()}  

data_copy['Label_Index'] = data_copy['Label'].map(reverse_label_map)

def calculate_weight(index):
    if index == 1:
        return 1
    elif 1.1 <= index <= 2.9:
        return 2.95
    elif 3 <= index <= 3.9:
        return 5.9
    elif 4 <= index <= 6:
        return 6
    else:
        return None  

data_copy['NILAI_SENTIMEN'] = data_copy['Label_Index'].apply(calculate_weight)

# print(data_copy)


print(data_copy[['Label','NILAI_SENTIMEN']])
data_copy.to_csv('data/hasil/main_data_17_OQ1_dashboard.csv', index=False, sep=';')

label_counts = data_copy['Label'].value_counts()
label_summary = pd.DataFrame(label_counts).reset_index()
label_summary.columns = ['Label', 'Count']

              Label  NILAI_SENTIMEN
0     sangat setuju             6.0
1     sangat setuju             6.0
2            setuju             6.0
3            setuju             6.0
4            setuju             6.0
...             ...             ...
2916         setuju             6.0
2917         setuju             6.0
2918         setuju             6.0
2919         setuju             6.0
2920         setuju             6.0

[2921 rows x 2 columns]


In [14]:
label_counts = data_copy['Label'].value_counts()

label_summary = pd.DataFrame(label_counts).reset_index()
label_summary.columns = ['Label', 'Count']
print(label_summary)

                 Label  Count
0               setuju   2393
1        sangat setuju    324
2         cukup setuju    140
3         tidak setuju     63
4  sangat tidak setuju      1
